In [6]:
import pandas as pd
import numpy as np
import glob
import laspy
#import open3d as o3d
import cv2

import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

from PIL import Image

In [7]:
def GetPathRelations(full_path_to_data):        
    ground_removed_image_paths = []
    laz_point_cloud_paths = []
        
    # Find full path to all images
    for path in glob.glob(full_path_to_data+'data/ImagesGroundRemovedSmall/*'):
        ground_removed_image_paths.append(path)
    
    # Find full path to all laz files
    for path in glob.glob(full_path_to_data+'data/LazFilesWithHeightRemoved/*'):
        laz_point_cloud_paths.append(path)
            
    ground_removed_image_paths.sort()
    laz_point_cloud_paths.sort()
    assert(len(ground_removed_image_paths)==len(laz_point_cloud_paths))
    return ground_removed_image_paths, laz_point_cloud_paths

In [8]:
def MaxMinNormalize(arr):
    return (arr - np.min(arr))/(np.max(arr)-np.min(arr))

def CastAllXValuesToImage(arr, x_pixels):
    return (MaxMinNormalize(arr))*x_pixels

def CastAllYValuesToImage(arr, y_pixels):
    return (1-MaxMinNormalize(arr))*y_pixels

In [10]:
all_path_relations = GetPathRelations("/home/frederik/data/TestData/")
path_tuples = list(zip(*all_path_relations))

transform_img_gray = transforms.Compose(
    [transforms.Resize((1001,1001)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Open images
trainingImages = []
labelImages = []
    
for path in path_tuples:
    image_path, laz_path = path
    
    # Image to training set
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    image = np.where(image >= 0, image, 0)
    image = image/np.max(image)
    image = (image*255).astype(np.uint8)
    x_pixels, y_pixels = image.shape
    
    image = Image.fromarray(image)
    trainingImages.append(transform_img_gray(image))
    
    # Generate labels 
    las = laspy.read(laz_path, laz_backend=laspy.compression.LazBackend.LazrsParallel)
    
    print(x_pixels, y_pixels)
    
    y_values = np.rint(CastAllXValuesToImage(las.X, y_pixels)).astype(np.int32)
    x_values = np.rint(CastAllYValuesToImage(las.Y, x_pixels)).astype(np.int32)
    
    powerline_mask = (las.classification == 14)
    x_powerline_values = x_values[powerline_mask]
    x_powerline_values = np.where(x_powerline_values < x_pixels, x_powerline_values, x_pixels-1)
    x_powerline_values = np.where(x_powerline_values >= 0, x_powerline_values, 0)
    
    y_powerline_values = y_values[powerline_mask]
    y_powerline_values = np.where(y_powerline_values < y_pixels, y_powerline_values, y_pixels-1)
    y_powerline_values = np.where(y_powerline_values >= 0, y_powerline_values, 0)
    
    labels = np.zeros((x_pixels, y_pixels)).astype(np.uint8)
    for i in range(len(x_powerline_values)):
        labels[x_powerline_values[i], y_powerline_values[i]] = 255
        
    
    linesP = cv2.HoughLinesP(
            labels, # Input edge image
            10, # Distance resolution in pixels
            np.pi/180, # Angle resolution in radians
            threshold=0, # Min number of votes for valid line
            minLineLength=0, # Min allowed length of line
            maxLineGap=10
    )
    lines_image = np.zeros_like(labels)
    # Draw the lines
    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv2.line(lines_image, (l[0], l[1]), (l[2], l[3]), (255,0,0), 3)
    
    
    # Create kernel
    kernel = np.ones((3, 3), np.uint8)
    lines_image = cv2.dilate(lines_image, kernel, iterations=1)
    
#     fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15,10))
#     ax0.set_title('Labels')
#     ax0.imshow(labels, cmap='gray')
#     ax1.set_title('Hough Line')
#     ax1.imshow(lines_image, cmap='gray')
#     ax0.axis('off')
#     ax1.axis('off')
#     plt.show()

    lines_image = Image.fromarray(lines_image)
    labelImages.append(transform_img_gray(lines_image))
    

    
X_train, X_test, Y_train, Y_test = train_test_split(trainingImages, labelImages, test_size=0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.15)

print(len(X_train), len(Y_train))
print(len(X_test), len(Y_test))
print(len(X_val), len(Y_val))


# Setting up sets for trainlodader and validation loader
training_set = []
for i in range(len(X_train)):
    training_set.append([X_train[i], Y_train[i]])
    
validation_set = []
for i in range(len(Y_val)):
    validation_set.append([X_val[i], Y_val[i]])
    
test_set = []
for i in range(len(Y_test)):
    test_set.append([X_test[i], Y_test[i]])

trainloader = torch.utils.data.DataLoader(training_set, batch_size=1, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(validation_set, batch_size=1, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=True, num_workers=4)

4095
1001 1001
4095
1001 1001
4095
1001 1001
4095
1001 1001
4095
1001 1001
4095
1001 1001
4095
1001 1001
4095
1001 1001
4095
1000 1000
4095
1001 1001
4095
1000 1000
4095
1001 1001
4095
1001 1001
4095
1001 1000
4095
996 1000
10 10
3 3
2 2


In [21]:
for i, data in enumerate(trainloader):
    # get the input
    inputs, labels = data
    print(inputs.shape)

torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])
torch.Size([1, 1, 1001, 1001])


In [18]:
class ConvNetRGB(nn.Module):
    def __init__(self):
        super(ConvNetRGB, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding = 1),
            nn.ReLU()).cuda()
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding = 1),
            nn.ReLU()).cuda()
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)).cuda()
        
        self.drop_out = nn.Dropout()
        
        self.down1 = nn.ConvTranspose2d(in_channels=256, out_channels=64,
                                        stride = 2, kernel_size=3, padding = 1, output_padding=1).cuda()
        
        self.down2 = nn.ConvTranspose2d(in_channels=64, out_channels=16,
                                        kernel_size=3, stride=1, padding=1).cuda()
        
        self.down3 = nn.ConvTranspose2d(in_channels=16, out_channels=1,
                                        kernel_size=3, stride=1, padding=1).cuda()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.drop_out(out)
        out = self.down1(out)
        out = self.down2(out)
        out = self.down3(out)
        return out

In [22]:
def ConvNetTraining(trainloader, valloader, Conv, lossFunction,learning_rate, epochs):
    model = Conv
    num_epochs = epochs
    criterion = lossFunction.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # loss arrays for figures
    TrainingLossArray = []
    ValidationLossArray = []
    
    early_stopping = 20
    notImproved = 0
    bestLoss = None
    bestModel = None
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            # get the input
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize            
            outputs = model(inputs.cuda())
            loss = criterion(outputs.cuda(), labels.cuda())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Appending the mean running loss
        TrainingLossArray.append(running_loss/i)
        
        print("Training loss: ", running_loss/i)
        
        # Finding validation loss
        validation_loss = 0
        for i, data in enumerate(valloader):
            # get the inputs
            inputs, labels = data
            
            #Calculates loss
            outputs = model(inputs.cuda())            
            loss = criterion(outputs.cuda(), labels.cuda())      
            validation_loss += loss.item()
        # Appending the mean validation loss
        ValidationLossArray.append(validation_loss/i)
        print("Validation loss: ", validation_loss/i)
        
        # Initialising params for early stopping
        if bestLoss == None:
            bestLoss = validation_loss
        
        # Checks for early stopping
        if validation_loss <= bestLoss:
            notImproved = 0
            bestLoss = validation_loss
            bestModel = model
            torch.save(bestModel, "bestModel.pth")
        else:
            notImproved +=1
        # Converges if the training has not improved for a certain amount of iterations
        if notImproved >= early_stopping:
            break
        
    return bestModel, ValidationLossArray, TrainingLossArray

In [23]:
bestModel, ValidationLossArray, TrainingLossArray = ConvNetTraining(trainloader, valloader, ConvNetRGB(), nn.MSELoss(), 0.0001, 1000)

RuntimeError: CUDA out of memory. Tried to allocate 246.00 MiB (GPU 0; 3.94 GiB total capacity; 2.41 GiB already allocated; 195.81 MiB free; 2.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF